In [52]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import source.pipelineprocesser as plp

df = pd.read_csv("cancer.tsv", sep="\t")
print(df.shape)
y = df["lpsa"]
X = df.drop(["lpsa", "Unnamed: 0", "train"], axis=1)
print(hoge := X.columns)
X = X.values

print(y.shape, X.shape)
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
print(X.shape)

y = (y - np.mean(y)) / np.std(y)


def option():
    X, y = plp.make_dataset()
    y = plp.mean_value_imputation(X, y)

    O = plp.soft_ipod(X, y, 0.015)
    X, y = plp.remove_outliers(X, y, O)

    M = plp.marginal_screening(X, y, 5)
    X = plp.extract_features(X, M)

    M1 = plp.stepwise_feature_selection(X, y, 3)
    M2 = plp.lasso(X, y, 0.08)
    M = plp.union(M1, M2)
    return plp.make_pipeline(output=M)

pl = option()
M, O = pl(X, y)
print(M, O)
print(hoge[M])
print()

M, results = pl.inference(X, y, is_result=True, precision=1e-5)

print([r.p_value for r in results])
print(pl.inference(X, y, over_conditioning=True)[1])
print([2 * norm.cdf(-np.abs(r.stat)) for r in results])


(97, 11)
Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45'], dtype='object')
(97,) (97, 8)
(97, 8)
[0, 1, 4, 7] [38]
Index(['lcavol', 'lweight', 'svi', 'pgg45'], dtype='object')

[3.62989673874381e-08, 0.01847881174944166, 0.06980997352081553, 0.7369299275116836]
[0.000150491577944889, 0.20100807283242816, 0.05814979771097262, 0.7304913247221283]
[2.213581031408327e-11, 5.268747028860387e-05, 0.0013070871456503843, 0.38976707111507636]
